# Adaptive Boosting - AdaBoost

In [ ]:
# Predeciremos las especies de pingüinos de la longitud y profundidad de culmen.

import pandas as pd

penguins = pd.read_csv("../../../data/penguins/penguins_classification.csv")
culmen_columns = ["Culmen Length (mm)", "Culmen Depth (mm)"]
target_column = "Species"

data, target = penguins[culmen_columns], penguins[target_column]

In [ ]:
# Entrenamos un árbol de decisión poco profundo.
# Since it is shallow, it is unlikely to overfit and some of the training examples will even be misclassified.
# Dado que es superficial, es poco probable que sobrejuste y e inclusi algunas muestras de entrenamiento clasificarán erróneamente.

import seaborn as sns
from sklearn.tree import DecisionTreeClassifier

palette = ["tab:red", "tab:blue", "black"]

tree = DecisionTreeClassifier(max_depth=2, random_state=0)
tree.fit(data, target)

In [ ]:
# podemos predecir en el mismo conjunto de datos y verificar qué muestras se clasifican erróneamente.

import numpy as np

target_predicted = tree.predict(data)
misclassified_samples_idx = np.flatnonzero(target != target_predicted)
data_misclassified = data.iloc[misclassified_samples_idx]

In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import DecisionBoundaryDisplay

DecisionBoundaryDisplay.from_estimator(
    tree, data, response_method="predict", cmap="RdBu", alpha=0.5
)

# el conjunto de datos original
sns.scatterplot(data=penguins, x=culmen_columns[0], y=culmen_columns[1],
                hue=target_column, palette=palette)
# las muestras mal clasificadas
sns.scatterplot(data=data_misclassified, x=culmen_columns[0],
                y=culmen_columns[1], label="Misclassified samples",
                marker="+", s=150, color="k")

plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left")
_ = plt.title("Predicciones de árbol de decisión \ncon muestras mal clasificadas "
              "highlighted")

> Hemos mencionado que el Boosting se basa en crear un nuevo clasificador que intente corregir estas clasificaciones erróneas.
- En Scikit-Learn, los modelos tienen un parámetro `sample_weight` que le obliga a prestar más atención a las muestras con pesos más altos durante el entrenamiento.

Este parámetro se establece al llamar a `classifier.fit(X, y, sample_weight=weights)`.
- Usaremos este truco para crear un nuevo clasificador "descartando" todas las muestras clasificadas correctamente y solo considerando las muestras mal clasificadas.
- Se asignará un peso de 1 a las muestras mal clasificadas  y 0 a las muestras bien clasificadas.

In [ ]:
sample_weight = np.zeros_like(target, dtype=int)
sample_weight[misclassified_samples_idx] = 1

tree = DecisionTreeClassifier(max_depth=2, random_state=0)
tree.fit(data, target, sample_weight=sample_weight)

In [ ]:
DecisionBoundaryDisplay.from_estimator(
    tree, data, response_method="predict", cmap="RdBu", alpha=0.5
)
sns.scatterplot(data=penguins, x=culmen_columns[0], y=culmen_columns[1],
                hue=target_column, palette=palette)
sns.scatterplot(data=data_misclassified, x=culmen_columns[0],
                y=culmen_columns[1],
                label="Muestras previamente mal clasificadas",
                marker="+", s=150, color="k")

plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left")
_ = plt.title("Árbol de decisión cambiando los pesos de muestra")

In [ ]:
target_predicted = tree.predict(data)
newly_misclassified_samples_idx = np.flatnonzero(target != target_predicted)
remaining_misclassified_samples_idx = np.intersect1d(
    misclassified_samples_idx, newly_misclassified_samples_idx
)

print(f"Número de muestras previamente mal clasificadas y "
      f"aún mal clasificado: {len(remaining_misclassified_samples_idx)}")

> Sin embargo, estamos cometiendo errores en muestras previamente bien clasificadas.
- Por tanto, obtenemos entendemos que debemos ponderar las predicciones de cada clasificador de manera diferente, 
- muy probablemente usando el número de errores que cada clasificador está cometiendo.

In [ ]:
# Podríamos usar el error de clasificación para combinar ambos árboles.

ensemble_weight = [
    (target.shape[0] - len(misclassified_samples_idx)) / target.shape[0],
    (target.shape[0] - len(newly_misclassified_samples_idx)) / target.shape[0],
]
ensemble_weight

> El primer clasificador fue 94% preciso y el segundo 69%.
- Por tanto, al predecir una clase, debemos confiar en el primer clasificador un poco más que el segundo.
- Podríamos usar estos valores de precisión para ponderar las predicciones de cada modelo.

> El Boosting requiere una estrategia para combinar los modelos:
- Se necesita definir una forma de calcular los pesos que se asignarán a las muestras;
- Se debe asignar un peso a cada modelo al hacer predicciones.

In [ ]:
# Usaremos el clasificador AdaBoost y analizaremos los clasificadores de árbol de decisión subyacentes.

from sklearn.ensemble import AdaBoostClassifier

base_estimator = DecisionTreeClassifier(max_depth=3, random_state=0)
adaboost = AdaBoostClassifier(estimator=base_estimator,
                              n_estimators=3, algorithm="SAMME",
                              random_state=0)
adaboost.fit(data, target)

In [ ]:
for boosting_round, tree in enumerate(adaboost.estimators_):
    plt.figure()
    # Convertimos `data` en una matriz numpy
    DecisionBoundaryDisplay.from_estimator(
        tree, data.to_numpy(), response_method="predict", cmap="RdBu", alpha=0.5
    )
    sns.scatterplot(x=culmen_columns[0], y=culmen_columns[1],
                    hue=target_column, data=penguins,
                    palette=palette)
    plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left")
    _ = plt.title(f"Árbol de decisión entrenado en ronda {boosting_round}")

In [ ]:
print(f"Peso de cada clasificador: {adaboost.estimator_weights_}")

In [ ]:
print(f"Error de cada clasificador: {adaboost.estimator_errors_}")